In [1]:
import pandas as pd
import sys
import os
sys.path.append("../")
sys.path.append("../..")

# Data Ingestion Initialization

## DB Client 

In [2]:
from KETIPreDataIngestion.KETI_setting import influx_setting_KETI as ins
from KETIPreDataIngestion.data_influx import influx_Client

DBClient = influx_Client.influxClient(ins)
db_list = DBClient.get_DBList()

In [3]:
db_list

['energy_solar',
 'traffic_seoul_subway',
 'traffic_seoul_bus',
 'test',
 'energy_electronic_power',
 'test_bio_covid',
 'test_indoor_high',
 'air_indoor_중학교',
 'traffic_seoul_bike',
 'traffic_public_parking',
 'energy_electricity_transaction',
 'energy_fire_power',
 'traffic_expressway',
 'farm_inner_air',
 'farm_outdoor_weather',
 'farm_outdoor_weather_clean',
 'farm_outdoor_air',
 'farm_outdoor_air_clean',
 'air_outdoor_에어코리아',
 'energy_water_discharge',
 'energy_wind_power',
 'city_exhibition_entrance_status',
 'bio_covid_vaccinations',
 'air_indoor_유치원',
 'air_indoor_요양원',
 'air_indoor_아파트',
 'air_indoor_도서관',
 'air_indoor_경로당',
 'air_indoor_초등학교',
 'air_indoor_체육시설',
 'air_indoor_어린이집',
 'air_indoor_고등학교',
 'air_outdoor',
 'bio_covid_infected_world',
 'bio_covid_infected_korea',
 'bio_test_infected',
 'keti_rtd',
 'culture_subway',
 'farm_strawberry_knu',
 'weather_korean_nws',
 'finance_korean_stock',
 'farm_strawberry_gunwi',
 'farm_strawberry_awon',
 'farm_strawberry_jinan']

## DB MS Frequency

In [10]:
db_name ='air_indoor_중학교'
ms_name = DBClient.measurement_list(db_name)[0]

In [11]:
freq = DBClient.get_freq(db_name, ms_name)
freq

TypeError: __init__() missing 1 required positional argument: 'data'

## DB and Measurement name setting

In [12]:
db_name  = 'finance_korean_stock'

In [13]:
ms_list = DBClient.measurement_list(db_name)
ms_name = ms_list[0]
print(ms_name)

stock


# Data Ingestion Methods

## pure data ingestion

### Ingest Full Data

In [14]:
print(db_name, ms_name)

finance_korean_stock stock


In [15]:
feature_list = DBClient.get_fieldList(db_name, ms_name)
number = "100000"
data = DBClient.get_datafront_by_num(number, db_name, ms_name)
data.head()

,adj_close_value,close_value,company,country,exchange,high_value,industry,low_value,open_value,ticker,volume_value
time,,,,,,,,,,,
2021-12-01 00:00:00+00:00,804.000000,804.000000,ASML 홀딩 NV ADR,US,NASDAQ,832.679993,반도체 및 반도체 장비,802.440002,812.559998,ASML,1200924.0
2021-12-02 00:00:00+00:00,2330.000000,2330.000000,성창기업지주,KR,KOSPI,2375.000000,종이와목재,2175.000000,2205.000000,000180.KS,262641.0
2021-12-16 00:00:00+00:00,755.000000,755.000000,ASML 홀딩 NV ADR,US,NASDAQ,799.840027,반도체 및 반도체 장비,753.618408,799.000000,ASML,1006869.0
2021-12-17 00:00:00+00:00,85.900002,85.900002,노바티스 AG ADR,US,NYSE,86.629997,제약,85.610001,86.300003,NVS,5301561.0
2021-12-20 00:00:00+00:00,5310.000000,5310.000000,AJ네트웍스,KR,KOSPI,5470.000000,도로와철도운송,5280.000000,5410.000000,095570.KS,67345.0


In [16]:
first = DBClient.get_first_time(db_name, ms_name)
last = DBClient.get_last_time(db_name, ms_name)
print(first, last)

                      adj_close_value  close_value         company country  \
time                                                                         
2021-12-01T00:00:00Z            804.0        804.0  ASML 홀딩 NV ADR      US   

                     exchange  high_value      industry   low_value  \
time                                                                  
2021-12-01T00:00:00Z   NASDAQ  832.679993  반도체 및 반도체 장비  802.440002   

                      open_value ticker  volume_value  
time                                                   
2021-12-01T00:00:00Z  812.559998   ASML     1200924.0  
                      adj_close_value  close_value company country exchange  \
time                                                                          
2021-12-20T00:00:00Z          34800.0      34800.0   휠라홀딩스      KR    KOSPI   

                      high_value      industry  low_value  open_value  \
time                                                                    

In [17]:
full_data = DBClient.get_data(db_name, ms_name)

In [18]:
query_end_time = last
query_start_time = first
bind_params = {'end_time':query_end_time, 'start_time': query_start_time}
time_data = DBClient.get_data_by_time(bind_params, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           adj_close_value  close_value         company  \
time                                                                      
2021-12-01 00:00:00+00:00       804.000000   804.000000  ASML 홀딩 NV ADR   
2021-12-02 00:00:00+00:00      2330.000000  2330.000000          성창기업지주   
2021-12-16 00:00:00+00:00       755.000000   755.000000  ASML 홀딩 NV ADR   
2021-12-17 00:00:00+00:00        85.900002    85.900002     노바티스 AG ADR   

                          country exchange   high_value      industry  \
time                                                                    
2021-12-01 00:00:00+00:00      US   NASDAQ   832.679993  반도체 및 반도체 장비   
2021-12-02 00:00:00+00:00      KR    KOSPI  2375.000000         종이와목재   
2021-12-16 00:00:00+00:00      US   NASDAQ   799.840027  반도체 및 반도체 장비   
2021-12-17 00:00:00+00:00      US     NYSE    86.629997            제약   

                             low_value   open_value     ticker  volume_value  
time                           

In [19]:
number="20"
time_data = DBClient.get_datafront_by_num(number, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           adj_close_value  close_value         company  \
time                                                                      
2021-12-01 00:00:00+00:00            804.0        804.0  ASML 홀딩 NV ADR   

                          country exchange  high_value      industry  \
time                                                                   
2021-12-01 00:00:00+00:00      US   NASDAQ  832.679993  반도체 및 반도체 장비   

                            low_value  open_value ticker  volume_value  
time                                                                    
2021-12-01 00:00:00+00:00  802.440002  812.559998   ASML     1200924.0  
                           adj_close_value  close_value         company  \
time                                                                      
2021-12-01 00:00:00+00:00            804.0        804.0  ASML 홀딩 NV ADR   

                          country exchange  high_value      industry  \
time                                    

In [20]:
days = 7 
bind_params = {'end_time': query_end_time,"days":str(days)+"d"}
day_data = DBClient.get_data_by_days(bind_params, db_name, ms_name)
print(day_data.head(), day_data.tail())

                           adj_close_value  close_value         company  \
time                                                                      
2021-12-16 00:00:00+00:00       755.000000   755.000000  ASML 홀딩 NV ADR   
2021-12-17 00:00:00+00:00        85.900002    85.900002     노바티스 AG ADR   
2021-12-20 00:00:00+00:00      5310.000000  5310.000000          AJ네트웍스   

                          country exchange   high_value      industry  \
time                                                                    
2021-12-16 00:00:00+00:00      US   NASDAQ   799.840027  반도체 및 반도체 장비   
2021-12-17 00:00:00+00:00      US     NYSE    86.629997            제약   
2021-12-20 00:00:00+00:00      KR    KOSPI  5470.000000       도로와철도운송   

                             low_value   open_value     ticker  volume_value  
time                                                                          
2021-12-16 00:00:00+00:00   753.618408   799.000000       ASML     1006869.0  
2021-12-17 00:00:00+0

In [21]:
tag_list = DBClient.get_tagList(db_name, ms_name)

In [22]:
tag_list

['company', 'country', 'exchange', 'industry', 'ticker']

In [23]:
tag_key = 'company'
tag_value='컴캐스트'

df = DBClient.get_TagGroupData(db_name, ms_name, tag_key, tag_value)

select * from "stock" WHERE "company"='컴캐스트'
                           adj_close_value  close_value company country  \
time                                                                      
2021-12-01 00:00:00+00:00        49.080002    49.080002    컴캐스트      US   
2021-12-16 00:00:00+00:00        48.840000    48.840000    컴캐스트      US   
2021-12-17 00:00:00+00:00        48.750000    48.750000    컴캐스트      US   

                          exchange  high_value  industry  low_value  \
time                                                                  
2021-12-01 00:00:00+00:00   NASDAQ   51.400002  미디어 및 출판  49.049999   
2021-12-16 00:00:00+00:00   NASDAQ   49.520000  미디어 및 출판  48.419998   
2021-12-17 00:00:00+00:00   NASDAQ   49.250000  미디어 및 출판  48.040001   

                           open_value ticker  volume_value  
time                                                        
2021-12-01 00:00:00+00:00   51.134899  CMCSA    20791588.0  
2021-12-16 00:00:00+00:00   48.549999  C

In [ ]:
tag_distinct_value = DBClient.get_TagValue(db_name, ms_name, tag_key)

In [ ]:
tag_distinct_value